In [ ]:
import numpy as np
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
from keras.models import Model
import tensorflow as tf
import pandas as pd 
from keras.layers import Input, LSTM, GRU, Dense, Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import matplotlib.pyplot as plt
import re
import pickle

In [ ]:
nor_train=open("/kaggle/input/test-dataset/normal.training.txt","r")
sim_train=open("/kaggle/input/test-dataset/simple.training.txt","r")
normal_train=nor_train.readlines()
simple_train=sim_train.readlines()

In [ ]:
def preprocess_sentence(w):
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)
  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
  w = w.strip()
  w = 'sssss ' + w + ' eeeee'
  return w

In [ ]:
BATCH_SIZE = 1000
EPOCHS = 64
NUM_SENTENCES = 1000
MAX_SENTENCE_LENGTH = 1200
MAX_NUM_WORDS = 4000
EMBEDDING_SIZE = 100

In [ ]:
input_sentences = []
output_sentences = []
output_sentences_inputs = []
i=0
k=0
while i <(NUM_SENTENCES):
  try:
    s1=normal_train[k]
  except:
    print(k)
    break
  if(len(s1)<MAX_SENTENCE_LENGTH):
    input_sentences.append(s1)
    i=i+1
    sentence = simple_train[k]
    output_sentences.append(sentence)
  k=k+1

In [ ]:
for i in range(len(input_sentences)):
    input_sentences[i]=preprocess_sentence(input_sentences[i])
for i in range(len(output_sentences)):
    output_sentences[i]=preprocess_sentence(output_sentences[i])
len(input_sentences)
print(input_sentences[:5])

In [ ]:
input_tokenizer = Tokenizer(num_words=15000,oov_token='zzzzz')
input_tokenizer.fit_on_texts(input_sentences)
input_integer_seq = input_tokenizer.texts_to_sequences(input_sentences)
word2idx_inputs = input_tokenizer.word_index
print('Total unique words in the input: %s' % len(word2idx_inputs))
max_input_len = max(len(sen) for sen in input_integer_seq)
print("Length of longest sentence in input: %g" % max_input_len)

In [ ]:
output_tokenizer = Tokenizer(filters='',num_words=15000,oov_token='zzzzz')
output_tokenizer.fit_on_texts(output_sentences + output_sentences_inputs)
output_integer_seq = output_tokenizer.texts_to_sequences(output_sentences)
word2idx_outputs = output_tokenizer.word_index
print('Total unique words in the output: %s' % len(word2idx_outputs))
num_words_output = len(word2idx_outputs) + 1
max_out_len = max(len(sen) for sen in output_integer_seq)
print("Length of longest sentence in the output: %g" % max_out_len)


In [ ]:
input_vocab=[]
output_vocab=[]
for key,val in word2idx_inputs.items():
    input_vocab.append(key)
for key,val in word2idx_outputs.items():
    output_vocab.append(key)
input_vocab=input_vocab[:15000]
output_vocab=output_vocab[:15000]

In [ ]:
input_tensor = pad_sequences(input_integer_seq, maxlen=max_input_len)
target_tensor = pad_sequences(output_integer_seq, maxlen=max_out_len, padding='post')

In [ ]:
from sklearn.model_selection import train_test_split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

In [ ]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 1000
steps_per_epoch = len(input_tensor_train)/BATCH_SIZE
embedding_dim = 100
units = 1024
vocab_inp_size = 15001
vocab_out_size = 15001
steps_per_epoch=int(steps_per_epoch)

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

In [ ]:
import pickle
b_file=open("/kaggle/input/embeddings-dict/data.pkl","rb")
embeddings_dict = pickle.load(b_file)
# print(embeddings_dict)

In [ ]:
embeddings_dict[0]=np.zeros([100])
embeddings_dict[1]=np.zeros([100])#Make this an array of 1s
embeddings_dict[2]=np.zeros([100])#Make this an array of -1s
# embeddings_dict[3]

In [ ]:
embedding_matrix_input = np.zeros((15001, embedding_dim))
for word, i in word2idx_inputs.items():
    embedding_vector = embeddings_dict.get(i)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix_input[i] = embedding_vector

In [ ]:
import pickle
b_file=open("/kaggle/input/embeddings-dict/data1.pkl","rb")
embeddings_dict = pickle.load(b_file)
# print(embeddings_dict)

In [ ]:
embeddings_dict[0]=np.zeros([100])
embeddings_dict[1]=np.zeros([100])#Make this an array of 1s
embeddings_dict[2]=np.zeros([100])#Make this an array of -1s
# embeddings_dict[3]

In [ ]:
embedding_matrix_output = np.zeros((15001, embedding_dim))
for word, i in word2idx_outputs.items():
    embedding_vector = embeddings_dict.get(i)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix_output[i] = embedding_vector

In [ ]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(
    vocab_size,
    embedding_dim,
    embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix_input),
    trainable=False,
)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
  def call(self, x, hidden):
    #x = embeddings_dict(x)
    x=self.embedding(x)
    output, state = self.gru(x, initial_state=hidden)
    return output, state
  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [ ]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
tester= tf.keras.layers.Embedding(vocab_inp_size, embedding_dim)
tester(example_input_batch)

In [ ]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)
  def call(self, query, values):
    query_with_time_axis = tf.expand_dims(query, 1)
    score = self.V(tf.nn.tanh(
        self.W1(query_with_time_axis) + self.W2(values)))
    attention_weights = tf.nn.softmax(score, axis=1)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)
    return context_vector, attention_weights

In [ ]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

In [ ]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(
    vocab_size,
    embedding_dim,
    embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix_output),
    trainable=False,
)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)
    self.attention = BahdanauAttention(self.dec_units)
  def call(self, x, hidden, enc_output):
    context_vector, attention_weights = self.attention(hidden, enc_output)
    x = self.embedding(x)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
    output, state = self.gru(x)
    output = tf.reshape(output, (-1, output.shape[2]))
    x = self.fc(output)
    return x, state, attention_weights

In [ ]:
decoder = Decoder(vocab_out_size, embedding_dim, units, BATCH_SIZE)
sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)
  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask
  return tf.reduce_mean(loss_)

In [ ]:
import os
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [ ]:
def train_step(inp, targ, enc_hidden):
  print("Enter")
  loss = 0
  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([output_tokenizer.word_index['sssss']] * BATCH_SIZE, 1)
    for t in range(1, targ.shape[1]):
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
      loss += loss_function(targ[:, t], predictions)
      dec_input = tf.expand_dims(targ[:, t], 1)
      print("a",end=" ")
  batch_loss = (loss / int(targ.shape[1]))
  variables = encoder.trainable_variables + decoder.trainable_variables
  gradients = tape.gradient(loss, variables)
  optimizer.apply_gradients(zip(gradients, variables))
  print("4 done")
  return batch_loss

In [ ]:
import time
EPOCHS = 1
for epoch in range(EPOCHS):
  start = time.time()
  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0
  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    print("out")
    total_loss += batch_loss
    print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  checkpoint.save(file_prefix = checkpoint_prefix)
  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

In [ ]:
import numpy as np
def evaluate(sentence):
  attention_plot = np.zeros((max_out_len, max_input_len))
  sentence = preprocess_sentence(sentence)
  inputs = [input_tokenizer.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_input_len,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)
  result = ''
  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden)
  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([output_tokenizer.word_index['sssss']], 0)
  for t in range(max_out_len):
    predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                         dec_hidden,
                                                         enc_out)
    attention_weights = tf.reshape(attention_weights, (-1, ))
    attention_plot[t] = attention_weights.numpy()
    predicted_id = tf.argmax(predictions[0]).numpy()
    result += output_tokenizer.index_word[predicted_id] + ' '
    if output_tokenizer.index_word[predicted_id] == 'eeeee':
      return result, sentence, attention_plot
    dec_input = tf.expand_dims([predicted_id], 0)
  return result, sentence, attention_plot

In [ ]:
def translate(sentence):
  result, sentence, attention_plot = evaluate(sentence)
  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))

In [ ]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
translate(u"the")